In [1]:
import pandas as pd
import numpy as np

from keras.models import load_model
from keras.utils import to_categorical
from scipy.stats import zscore

from os.path import join
from datetime import datetime

/home/jonah/.conda/envs/odc/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
weather_path = join('..','data','weather','precip_temp.csv')
weather_df = pd.read_csv(weather_path)
# Add normalized columns
weather_df[['TMIN-norm','TMAX-norm']] = weather_df[['TMIN','TMAX']] - (weather_df[['TMIN', 'TMAX']].sum().sum() \
                              / weather_df[['TMIN', 'TMAX']].size)
weather_df[['TMIN-norm','TMAX-norm']] /= (weather_df[['TMIN-norm', 'TMAX-norm']].abs().sum().sum() \
                              / weather_df[['TMIN-norm', 'TMAX-norm']].size)
weather_df['PRCP-norm'] = zscore(weather_df['PRCP'])

model_path = join('..','models','model.h5')
model = load_model(model_path)

In [106]:
start = datetime(2017, 12, 21, 2)
end = datetime(2017, 12, 28) # end is not inclusive
dates = pd.date_range(start, end)
inputs = pd.DataFrame({'DATE': dates})
inputs['DATE'] = [dt.strftime('%Y-%m-%d') for dt in inputs['DATE']]
inputs = inputs.merge(weather_df[['DATE','TMIN-norm','TMAX-norm','PRCP-norm']], left_on='DATE', right_on='DATE')
for d in range(7):
    inputs['weekday-%i' % d] = [int(datetime.strptime(date, '%Y-%m-%d').weekday()==d) for date in inputs['DATE']]
inputs['Year %'] = [datetime.strptime(date, '%Y-%m-%d').timetuple()[7] / 365 for date in inputs['DATE']]
weather_x = inputs[['TMIN-norm', 'TMAX-norm', 'PRCP-norm']].values
day_x = inputs.filter(regex='weekday-').values
date_x = inputs[['Year %']]
predictions = model.predict([weather_x, day_x, date_x])
output = pd.DataFrame(index=dates)
output['# Clients'] = predictions[0]
output['# Sessions'] = predictions[1]

output['# Clients'].to_csv('Clients per day.csv', header=False)
output['# Sessions'].to_csv('Number of sessions over time.csv', header=False)

hours = pd.date_range(start, end, freq='4H')
usage = pd.DataFrame(index=hours)
usage['usage'] = 0
usage['usage'].to_csv('Usage over time.csv', header=False)

,# Clients,# Sessions
2017-12-21 02:00:00,59.511494,433.775421
2017-12-22 02:00:00,70.767860,517.815308
2017-12-23 02:00:00,73.985970,540.539124
2017-12-24 02:00:00,58.668018,426.571625
2017-12-25 02:00:00,64.165764,468.070038
2017-12-26 02:00:00,64.735191,472.631989
2017-12-27 02:00:00,77.029022,565.064331
